In [12]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [13]:
def scrap(page_number):
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }
    url = f'https://kolesa.kz/cars/toyota/?page={page_number}'
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'lxml')
        return soup
    else:
        return None

In [14]:
data = []
# number_of_pages = 190

In [ ]:
for i in range(200, 301):
    print(f"Parsing page {i}...")
    
    soup = scrap(i)
    
    if soup is None:
        print(f"Error on page {i} or page not found.")
        continue

    links = soup.find_all('a', class_='a-card__link')

    for link in links:
        title = link.text.strip()

        card = link.find_parent('div', class_='a-card')

        if card:
            price_tag = card.find('span', class_='a-card__price')
            price = price_tag.text.strip().replace('\xa0', ' ') if price_tag else None

            desc_tag = card.find('p', class_='a-card__description')
            description = desc_tag.text.strip() if desc_tag else None


            data.append({
                'title': title,
                'price': price,
                'description': description
            })
    
    time.sleep(1)

Parsing page 200...
Parsing page 201...
Parsing page 202...
Parsing page 203...
Parsing page 204...
Parsing page 205...
Parsing page 206...
Parsing page 207...
Parsing page 208...
Parsing page 209...
Parsing page 210...
Parsing page 211...
Parsing page 212...
Parsing page 213...
Parsing page 214...
Parsing page 215...
Parsing page 216...
Parsing page 217...
Parsing page 218...
Parsing page 219...
Parsing page 220...
Parsing page 221...
Parsing page 222...
Parsing page 223...
Parsing page 224...
Parsing page 225...
Parsing page 226...
Parsing page 227...
Parsing page 228...
Parsing page 229...
Parsing page 230...
Parsing page 231...
Parsing page 232...
Parsing page 233...
Parsing page 234...
Parsing page 235...
Parsing page 236...
Parsing page 237...
Parsing page 238...
Parsing page 239...
Parsing page 240...
Parsing page 241...
Parsing page 242...
Parsing page 243...
Parsing page 244...
Parsing page 245...
Parsing page 246...
Parsing page 247...
Parsing page 248...
Parsing page 249...


In [16]:
df = pd.DataFrame(data)

df = df.loc[df['title'] != ""]
df = df.reset_index(drop=True)

In [17]:
splits = df['description'].str.split(',', expand=True)

In [18]:
df['year'] = splits[0].str.replace('г.', '').str.strip()
df['body_type'] = splits[1].str.strip()
df['volume'] = splits[2].str.replace('л', '').str.strip()
df['fuel'] = splits[3].str.strip()
df['gearbox'] = splits[4].str.replace('КПП', '').str.strip()

In [19]:
df = df.drop(columns=['description'])
splits = df['body_type'].str.split(' ', expand=True)
df['type'] = splits[0]
df['body_type'] = splits[1]

In [20]:
df.shape

(2020, 8)

In [21]:
df.to_csv('kolesa_cars3.csv', index=False, encoding='utf-8-sig')